In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Layout
import ipywidgets as widgets
from scipy.stats import norm
import seaborn as sn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
font_size = 16

In [3]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

# Gaussian processes

<br>
PhD. Juan Ungredda

# Content

* Multivariate Gaussian Distribution
* Gaussian process
* Gaussian process regression
* Hyperparameter optimization
* Difficulties

# Univariate Gaussian Density

A random variable $x \sim \mathcal{N}(\mu, \sigma^2)$ presents a density function

$$
f_x(x) = \frac{1}{\sqrt{2\pi} \sigma} \exp \left( -\frac{1}{2} \left( \frac{x - \mu}{\sigma} \right)^2 \right)
$$

Where:
- $x$ is the random variable.
- $\mu$ is the mean, representing the central tendency of the distribution.
- $\sigma^2$ is the variance, determining the spread or dispersion of the distribution.


In [4]:
def plot_gaussians(mu, var):
    sn.set_style("darkgrid")
    x = np.linspace(-3.5,
                3.5, 100)
    y = norm.pdf(x)
    standard_normal = pd.DataFrame({"x": x, "z(x)": y})
    sn.lineplot(data = standard_normal, x="x", y="z(x)",label="$\mathcal{N}(0,1)$")
    y = norm.pdf(x, mu, np.sqrt(var))
    non_standard_normal = pd.DataFrame({"x": x, "f(x)": y})
    sn.lineplot(data = non_standard_normal, x="x",  y="f(x)", label="$\mathcal{N}( %s, %s )$" % (mu,var))
    plt.ylabel(r'$f_x$', size=font_size)
    plt.xlabel(r"$X$", size=font_size)
    plt.legend()

In [5]:
hide_toggle(for_next=True)

In [6]:
widget = interact(plot_gaussians,
         mu=FloatSlider(value=0, min=-5, max=5, step=1, description="$\mu$:", layout=Layout(width='50%')),
         var=FloatSlider(value=1, min=0.1, max=5, step=1, description='$\sigma^2$:', layout=Layout(width='50%')))

interactive(children=(FloatSlider(value=0.0, description='$\\mu$:', layout=Layout(width='50%'), max=5.0, min=-…

# Multivariate Gaussian Distribution

A random variable $x \sim \mathcal{N}( \boldsymbol{\mu}, \boldsymbol{\Sigma})$ presents a density function

$$ f_x(\mathbf{x}) = \frac{1}{(2\pi)^{d/2} |\boldsymbol{\Sigma}|^{1/2}} \exp\left(-\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu})\right) $$

Where:
- $ \mathbf{x} $ is a $ d $-dimensional vector representing the random variables.
- $ \boldsymbol{\mu} $ is the mean vector, representing the expected value of each random variable.
- $ \boldsymbol{\Sigma} $ is the covariance matrix, representing the relationships between the random variables.



In [7]:
def plot_2d_gaussian_density(mu1, mu2, var_x1, var_x2, cov):
    sn.set_style("darkgrid")
    mu = [mu1, mu2]  # mean
    Sigma = [[var_x1, cov], [cov, var_x2]]  # covariance matrix
    # Generate grid points for x and y
    x = np.linspace(-3.5, 3.5, 100)
    y = np.linspace(-3.5, 3.5, 100)
    X, Y = np.meshgrid(x, y)
    
    # Calculate the bivariate normal PDF for each pair of (x, y) values
    pos = np.dstack((X, Y))
    Z = multivariate_normal.pdf(pos, mean=mu, cov=Sigma)
    
    # Plot bivariate normal PDF
    plt.contour(X, Y, Z)
    
    plt.xlabel('$X_1$', size=font_size)
    plt.ylabel('$X_2$', size=font_size)
    plt.title('Bivariate Normal Distribution')

def plot_gaussian_density(mu1, mu2, var_x1, var_x2, cov):
    plot_2d_gaussian_density(mu1, mu2, var_x1, var_x2, cov)
    plt.show()

In [8]:
hide_toggle(for_next=True)

In [9]:
widget = interact(plot_gaussian_density,
         mu1=FloatSlider(value=0, min=-5, max=5, step=1, description='$\mu_1$:', layout=Layout(width='50%')),
         mu2=FloatSlider(value=0, min=-5, max=5, step=1, description='$\mu_2$:', layout=Layout(width='50%')),
         var_x1=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='$Var(x_1)$:', layout=Layout(width='50%')),
         var_x2=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='$Var(x_2)$:', layout=Layout(width='50%')),
         cov=FloatSlider(value=0.5, min=-5, max=5, step=0.1, description='$Cov(x_1, x_2)$:', layout=Layout(width='50%')))

interactive(children=(FloatSlider(value=0.0, description='$\\mu_1$:', layout=Layout(width='50%'), max=5.0, min…

# Covariance Matrix

For $ n $ random variables $ X_1, X_2, \ldots, X_n $, the multivariate covariance matrix $ \Sigma $ is defined as:

$$
\Sigma = \begin{bmatrix} 
\text{Var}(x_1) & \text{Cov}(x_1, x_2) & \ldots & \text{Cov}(x_1, x_n) \\
\text{Cov}(x_2, x_1) & \text{Var}(x_2) & \ldots & \text{Cov}(x_2, x_n) \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}(x_n, x_1) & \text{Cov}(x_n, x_2) & \ldots & \text{Var}(x_n)
\end{bmatrix}
$$

A covariance matrix $\Sigma$ must satisfy the following properties:

1. **Symmetry**: $ \text{cov}(x, x') = \text{cov}(x', x) $ for all $ x $ and $ x' $.
2. **Positive Semi-definiteness**: For any finite collection of points $ x_1, x_2, \ldots, x_n $, the covariance matrix $ \Sigma $ defined by $ \Sigma_{ij} = \text{cov}(x_i, x_j) $ is positive semi-definite.


## Positive Semi-Definite Matrix and Covariance Matrix

#### Quadratic Form

A quadratic form is a homogeneous polynomial of degree two in a number of variables. The quadratic form associated with a matrix $A$ and a vector $x$ is given by 

$$x^T A x$$

- **Postive semi-definite**: $x^T A x \geq 0$ for any vector $x \neq 0$.

# Exercise: Positive Semi-Definite Matrix and Covariance Matrix

<br>
<br>

**Part a** Prove that every covariance matrix is positive-semidefinite.

<br>
<br>

**Part b** Show that the following matrix is not positive-semidefinite $$
\begin{bmatrix}
1 & -2 \\
-2 & 1 \\
\end{bmatrix}
$$

<br>
<br>

**Solution:**

**Part a**:

$$
Q(y) = y^T \Sigma y = y^T \mathbb{E}[(\mathbf{x} -\mathbf{\mu})(\mathbf{x} -\mathbf{\mu})^T ]y = \text{Var}[y^T (\mathbf{X} - \boldsymbol{\mu})] \geq 0 
$$

**Part b**: consider $\mathbf{x} = \begin{bmatrix}
1 \\
1 \\
\end{bmatrix}$ or $\mathbf{x} = \begin{bmatrix}
-1 \\
1 \\
\end{bmatrix}$

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import ipywidgets as widgets
from IPython.display import display

# Define symbolic variables
x, y = sp.symbols('x y')

# Define a grid of points
x_vals = np.linspace(-3, 3, 100)
y_vals = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = np.zeros_like(X)

# Function to update matrix values
def update_matrix(a11,a22, a12, a21,):
    A = np.array([[a11,a12], [a21, a22]])
    update_plot(A)


# Define the quadratic form equation
def quadratic_form(A):
    return sp.expand((sp.Matrix([x, y]).T @ sp.Matrix(A) @ sp.Matrix([x, y]))[0])

# Function to update plot
def update_plot(A):
    Z = np.zeros_like(X)
    for i in range(len(x_vals)):
        for j in range(len(y_vals)):
            vec = np.array([X[i, j], Y[i, j]])
            Z[i, j] = np.dot(vec.T, np.dot(np.array(A), vec))

    # Plot the result using contour plot in Matplotlib
    plt.figure(figsize=(8, 6))
    plt.contour(X, Y, Z, levels=[1, 2, 3, 4, 5])  # Change levels as needed
    plt.xlabel('x')
    plt.ylabel('y')

    # Include the quadratic form equation in the title with LaTeX formatting
    plt.title(r'Quadratic Form: $' + sp.latex(quadratic_form(A)) + r'$')

    # Print the current matrix values inside the plot
    matrix_text = np.array2string(A, formatter={'float_kind':lambda x: "{:.1f}".format(x)})
    plt.text(0.05, 0.95, r'Matrix values:', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.05, 0.85, matrix_text, fontsize=12, transform=plt.gca().transAxes)

    plt.axis('equal')
    plt.grid(True)

In [11]:
hide_toggle(for_next=True)

In [12]:
# Create text input widgets for matrix values
a11_text = widgets.FloatText(value=1, description=r'$A_{11}$', layout=Layout(width='50%'))
a22_text = widgets.FloatText(value=1, description=r'$A_{22}$', layout=Layout(width='50%'))
a12_text = widgets.FloatText(value=0, description=r'$A_{12}$', layout=Layout(width='50%'))
a21_text = widgets.FloatText(value=0, description=r'$A_{21}$', layout=Layout(width='50%'))


# Use interact to link the text input widgets with the update_matrix function
widgets.interactive(update_matrix, a11=a11_text, a22=a22_text, a21=a21_text, a12=a12_text)

interactive(children=(FloatText(value=1.0, description='$A_{11}$', layout=Layout(width='50%')), FloatText(valu…

# Conditioning

<br>
Given the two random vectors $\mathbf{x}_{A}$ and $\mathbf{x}_{B}$, the conditional probability of $\mathbf{x}_{A}$ is defined as,

<br>

$$
p(\mathbf{x}_{A}|\mathbf{x}_{B}) = \frac{p(\mathbf{x}_{A}, \mathbf{x}_{B})}{p(\mathbf{x}_{B})}
$$

defined for $p(\mathbf{x}_{B}) > 0$

## Exercise: Gaussian Conditioning

Assume an n-dimensional random vector has a normal distribution,

$$
N(\left[\begin{array}{c}
{\bf x} \\
{\bf y}
\end{array}\right], \left[\begin{array}{c}
\mu_X \\
\mu_Y
\end{array}\right], \left[\begin{array}{cc}
A & C \\
C^T & B
\end{array}\right]) 
$$

where $ {\bf x}_1 $ and $ {\bf x}_2 $ are two subvectors of respective dimensions $ p $ and $ q $ with $ p+q=n $. Then, conditional distribution of $ {\bf y} $ given $ {\bf x} $ is also normal with mean vector

$$
\mu_{\bf y \vert \bf x} = \mu_Y + C^TA^{-1}({\bf x} - \mu_X)
$$

and covariance matrix

$$
\Sigma_{\bf y \vert \bf x} = B - C^TA^{-1}C
$$

**Proof:**

The joint density of $ {\bf x} $ is:

$$
p({\bf x},{\bf y}) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}\exp\left[-\frac{1}{2}Q(\tilde{\bf x})\right]
$$

where $ Q $ is defined as

$$
Q(\tilde{\bf x}) = (\tilde{\bf x}-\tilde{\mu})^T\Sigma^{-1}(\tilde{\bf x}-\tilde{\mu}) = [({\bf x}-\mu_X)^T, ({\bf y}-\mu_Y)^T] \left[\begin{array}{cc}
A & C \\
C^T&B
\end{array}\right] \left[\begin{array}{c}
{\bf x}-\mu_X \\
{\bf y}-\mu_Y
\end{array}\right]
$$

Here we have assumed

$$
\Sigma^{-1} = \left[\begin{array}{cc}
\tilde{A} & \tilde{C} \\
\tilde{C}^T&\tilde{B}
\end{array}\right]
$$

where

$$
\tilde{A} = (A-CB^{-1}C^T)^{-1}C^TA^{-1}
$$

$$
\tilde{B} = (B-C^TA^{-1}C)^{-1}CB^{-1}
$$

$$
\tilde{C} = -A^{-1}C(B-C^TA^{-1}C)^{-1} = \tilde{C}^T
$$

Substituting into $ Q(\tilde{\bf x}) $ to get:

$$
Q(\tilde{\bf x}) = ({\bf x}-\mu_X)^T A^{-1} ({\bf x}-\mu_X) + [({\bf y}-\mu_Y)-C^TA^{-1}({\bf x}-\mu_X)]^T(B-C^TA^{-1}C)^{-1}[({\bf y}-\mu_Y)-C^TA^{-1}({\bf x}-\mu_X)]
$$

Now the joint distribution can be written as:

$$
p({\bf x},{\bf y}) = \frac{1}{(2\pi)^{n/2}|A|^{1/2}}\exp\left[-\frac{1}{2}Q(\tilde{\bf x})\right] = N({\bf x}|\mu_X,A) \cdot N({\bf y}|b,M)
$$

The conditional distribution of $ {\bf y} $ given $ {\bf x} $ is

\begin{align*}
p({\bf y}\vert{\bf x}) &= \frac{p({\bf x},{\bf y})}{p({\bf x})} \\
&= \frac{1}{(2\pi)^{q/2}|M|^{1/2}}\exp\left[-\frac{1}{2}({\bf y}-b)^T M^{-1}({\bf y}-b)\right]
\end{align*}

with

$$
b = \mu_Y + C^TA^{-1}({\bf x}-\mu_X)
$$

$$
M = B - C^TA^{-1}C
$$

Consider $n = 2$, then,

$$
b = \mu_Y + \frac{Cov(x,y)}{Var(x)}({x}-\mu_X)
$$

$$
M =  Var(y) - \frac{Cov(x,y)^2}{Var(x)}
$$

In [13]:

def plot_conditional_bivariate_normal_y_given_x(x_value, corr, varx):
    sn.set_style("darkgrid")
    mu = [0, 0]  # mean
    sigma = [[varx, corr], [corr, 1]]  # covariance matrix
    # Define grid points for y
    y = np.linspace(-3.5, 3.5, 100)
    
    mu_y_given_x = mu[1] + sigma[1][0] / sigma[0][0] * (x_value - mu[0])
    sigma_y_given_x = sigma[1][1] - sigma[1][0] / sigma[0][0] * sigma[0][1]
    
    
    # Calculate the conditional bivariate normal PDF
    conditional_pdf = multivariate_normal.pdf(y, mean=mu_y_given_x, cov=sigma_y_given_x)
    
    
    # Plot conditional bivariate normal PDF
    plt.plot(conditional_pdf, y)
    plt.xlabel('Conditional Probability')
    plt.title(f'Conditional Probability given X = {x_value}')

def plot_bivariate_normal_with_line(x_value, corr, varx):
    sn.set_style("darkgrid")
    mu = [0, 0]  # mean
    sigma = [[varx, corr], [corr, 1]]  # covariance matrix
    def conditional_mean_function(x, positive_corr = True):
        if positive_corr:
            return mu[1] + sigma[1][0] / sigma[0][0] * (x - mu[0])
        else:
            return mu[1] - sigma[1][0] / sigma[0][0] * (x - mu[0])
    # Generate grid points for x and y
    x = np.linspace(-3.5, 3.5, 100)
    y = np.linspace(-3.5, 3.5, 100)
    X, Y = np.meshgrid(x, y)
    
    # Calculate the bivariate normal PDF for each pair of (x, y) values
    pos = np.dstack((X, Y))
    Z = multivariate_normal.pdf(pos, mean=mu, cov=sigma)
    
    # Plot bivariate normal PDF
    plt.contour(X, Y, Z)
    
    # Plot vertical line for conditioning on X = x_value
    plt.axvline(x=x_value, color='r', linestyle='--', label=f'X = {x_value}')
    
    # plot ellipsoid axis
    positive_pos_axis = conditional_mean_function(x, positive_corr = True)
    
    plt.plot(x, positive_pos_axis, color="salmon", linestyle="--",label="$\mu_{y/x}(x)$")
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.title('Bivariate Normal Distribution')

def conditioning_normal_distribution(x_value, corr, varx):
    
    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Plot bivariate normal distribution with conditioning line
    plt.subplot(1, 2, 1)
    plot_bivariate_normal_with_line(x_value, corr, varx)

    # Plot conditional probability given X = x_value
    plt.subplot(1, 2, 2)
    plot_conditional_bivariate_normal_y_given_x( x_value, corr, varx)

#     plt.tight_layout()


In [14]:
hide_toggle(for_next=True)

In [15]:
style = {'handle_color': 'white', "value":"white"}
widget = interact(conditioning_normal_distribution,
         x_value=FloatSlider(value=0, min=-5, max=5, step=0.3, description='$X$:', layout=Layout(width='50%'), style= style),
        corr=FloatSlider(value=0, min=-2, max=2, step=0.1, description='$Cov(x, y)$:', layout=Layout(width='50%'), style= style),
        varx=FloatSlider(value=1, min=1e-3, max=5, step=0.1, description='$Var(x)$:', layout=Layout(width='50%'), style= style))

interactive(children=(FloatSlider(value=0.0, description='$X$:', layout=Layout(width='50%'), max=5.0, min=-5.0…

# Marginalisation

<br>

Given the two random vectors $\mathbf{x}_{A}$ and $\mathbf{x}_{B}$, the marginal probability of $\mathbf{x}_{A}$ is given by,

<br>
<br>

$$
p(\mathbf{x}_{A}) = \int p(\mathbf{x}_{A}, \mathbf{x}_{B}) d\mathbf{x}_{B}
$$

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

def plot_marginalized_distribution_x(mu, sigma):
    sns.set_style("darkgrid")
    
    # Define grid points for x
    x = np.linspace(-3.5, 3.5, 100)
    
    # Calculate the marginal distribution of X
    marginal_pdf = []
    for xi in x:
        marginal_pdf.append(multivariate_normal.pdf(xi, mean=mu[0], cov=sigma[0][0]))
    
    # Plot marginalized distribution of X (rotated 90 degrees clockwise)
    plt.plot(x, marginal_pdf)  # Interchange X and Y
    
    plt.title('Marginalized Distribution of X')

def plot_marginalized_distribution_y(mu, sigma):
    sns.set_style("darkgrid")
    
    # Define grid points for y
    y = np.linspace(-3.5, 3.5, 100)
    
    # Calculate the marginal distribution of Y
    marginal_pdf = []
    for yi in y:
        marginal_pdf.append(multivariate_normal.pdf(yi, mean=mu[1], cov=sigma[1][1]))
    
    # Plot marginalized distribution of Y (rotated 180 degrees clockwise and flipped)
    plt.plot(marginal_pdf, y)  
    
    plt.title('Marginalized Distribution of Y')

def plot_bivariate_normal_without_line(mu, sigma):
    sns.set_style("darkgrid")
    # Generate grid points for x and y
    x = np.linspace(-3.5, 3.5, 100)
    y = np.linspace(-3.5, 3.5, 100)
    X, Y = np.meshgrid(x, y)
    
    # Calculate the bivariate normal PDF for each pair of (x, y) values
    pos = np.dstack((X, Y))
    Z = multivariate_normal.pdf(pos, mean=mu, cov=sigma)
    
    # Plot bivariate normal PDF
    plt.contour(X, Y, Z)
    
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Bivariate Normal Distribution')

    
def plot_marginalization_gaussian(mu1, mu2, sigma_x, sigma_y, cov):
    mu = [mu1, mu2]  # mean
    Sigma = [[sigma_x, cov], [cov, sigma_y]]  # covariance matrix

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(8, 8), gridspec_kw={'width_ratios': [.75, .25], 'height_ratios': [.25, .75]})

    plt.subplot(2, 2, 1)
    plot_marginalized_distribution_x(mu, Sigma)

    axes[0, 1].remove()

    plt.subplot(2, 2, 3)
    plot_bivariate_normal_without_line(mu, Sigma)

    plt.subplot(2, 2, 4)
    plot_marginalized_distribution_y(mu, Sigma)

    plt.tight_layout()


# Exercise: Gaussian Marginalisation

<br>
<br>

Let $\mathbf{x}$ and $\mathbf{y}$ be jointly Gaussian random vector with dimension m and n, respectively.

$$
\begin{bmatrix}
\mathbf{x} \\
\mathbf{y}
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
\mu_{X}\\
\mu_{Y}
\end{bmatrix},
\begin{bmatrix}
A & C \\
C^T & B
\end{bmatrix}
\right)
$$

show that $x \sim \mathcal{N}(\mu_{X}, A)$

<br>
<br>
<br>
<br>
<br>
<br>
$\textbf{Solution}:$

$$
\mathbf{x} = A \begin{bmatrix}
\mathbf{x} \\
\mathbf{y}
\end{bmatrix} = A \bigg(\begin{bmatrix}
\mu_{X}\\
\mu_{Y}
\end{bmatrix} + BZ\bigg) = A \begin{bmatrix}
\mu_{X}\\
\mu_{Y}
\end{bmatrix} + ABZ
$$

where,

$$
A = [I_{m, m} , \mathbf{0}_{m, n}]
$$

Therefore, $\mathbf{x}$ is normally distributed with $\mathbb{E}[\mathbf{x}]=A \begin{bmatrix}
\mu_{X}\\
\mu_{Y}
\end{bmatrix}=\mu_{X}$ and $Cov(\mathbf{x})=A \begin{bmatrix}
A & C \\
C^T & B
\end{bmatrix} A^T=A$.

In [17]:
hide_toggle(for_next=True)

In [18]:
style = {'handle_color': 'white', "value":"white"}
widget = interact(plot_marginalization_gaussian,
         mu1=FloatSlider(value=0, min=-5, max=5, step=1, description='Mean1:', layout=Layout(width='50%'), style= style),
         mu2=FloatSlider(value=0, min=-5, max=5, step=1, description='Mean2:', layout=Layout(width='50%'), style= style),
         sigma_x=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='$\sigma_{x}$:', layout=Layout(width='50%'), style= style),
         sigma_y=FloatSlider(value=1, min=0.1, max=5, step=0.1, description='$\sigma_{y}$:', layout=Layout(width='50%'), style= style),
         cov=FloatSlider(value=0.5, min=-5, max=5, step=0.1, description='$\sigma_{xy} = \sigma_{yx}$::', layout=Layout(width='50%'), style= style))

interactive(children=(FloatSlider(value=0.0, description='Mean1:', layout=Layout(width='50%'), max=5.0, min=-5…

# Gaussian Normal Samples

Given a Cholesky decomposition of the covariance matrix to obtain the lower triangular matrix $L$,

$$
\Sigma = L L^T
$$

Then, you can generate samples from a standard normal distribution and transform them using the mean vector and the Cholesky decomposition of the covariance matrix:

$$
\mathbf{x} = \mu + L \mathbf{z}
$$

where $\mathbf{z} \sim N(\mathbf{0}, I)$. For a single dimension we have,

$$
x = \mu + \sigma z
$$

In [19]:
def plot_bivariate_gaussian_with_samples(num_samples, sigma_x, sigma_y, cov):
    sn.set_style("darkgrid")
    plot_2d_gaussian_density(0, 0, sigma_x, sigma_y, cov)
    samples = plot_random_normal_samples(num_samples, sigma_x, sigma_y, cov)
    plt.scatter(samples[:, 0], samples[:, 1])
    plt.show()
    
def plot_random_normal_samples(num_samples, sigma_x, sigma_y, cov):
    mean = [0, 0]  # Mean of the distribution
    np.random.seed(0)
    covariance_matrix = np.array([[sigma_x, cov], [cov, sigma_y]])  # Covariance matrix
    inverse_covariance_matrix = np.linalg.inv(covariance_matrix)  # Inverse of covariance matrix

    # Generate samples
    samples = np.random.randn(num_samples, 2)  # Generate samples from standard normal distribution

    # Transform samples using the mean and covariance matrix
    samples_transformed = mean + np.dot(samples, np.linalg.cholesky(covariance_matrix).T)
    
    # Print the generated samples
    return samples_transformed

In [20]:
hide_toggle(for_next=True)

In [21]:
# Create text input widgets for matrix values

num_samples_text = widgets.IntText(value=3, description=r'$n$', layout=Layout(width='50%'))

a11_text = FloatSlider(value=3, min=1e-3, max=5, step=0.5, description=r'$Var(x_1)$', layout=Layout(width='60%'), style= style)
a22_text = FloatSlider(value=3, min=1e-3, max=5, step=0.5, description=r'$Var(x_2)$', layout=Layout(width='60%'), style= style)
cov_text = FloatSlider(value=0, min=-4, max=4, step=0.5, description=r'$Cov(x_1, x_2)$', layout=Layout(width='60%'), style= style)

# Use interact to link the text input widgets with the update_matrix function
widgets.interactive(plot_bivariate_gaussian_with_samples, 
                    num_samples=num_samples_text,
                    sigma_x= a11_text, 
                    sigma_y=a22_text, 
                    cov=cov_text)

interactive(children=(IntText(value=3, description='$n$', layout=Layout(width='50%')), FloatSlider(value=3.0, …

# Summary


Let $\bf x$ and $\bf y$ be jointly Gaussian random vectors,

$$
\begin{bmatrix}
\bf x \\
\bf y
\end{bmatrix} \sim \mathcal{N}\left(\mu = \begin{bmatrix}
\mu_x \\
\mu_y
\end{bmatrix}, \Sigma = \begin{bmatrix}
A & C \\
C^T & B
\end{bmatrix}\right)
$$

### Conditioning 

$$
\mathbf{ x} \sim \mathcal{N}(\mu_x, A)
$$

### Marginalisation

$$
\mathbf{ x}| \mathbf{ y} \sim \mathcal{N}\left(\mu_x + CB^{-1}(\mathbf{ y} - \mu_y), A - CB^{-1}C^T\right)
$$

### Sampling


$$
\begin{bmatrix}
\bf x \\
\bf y
\end{bmatrix}  = \mu + L \mathbf{z} \text{, where } \Sigma = L L^T
$$

## Gaussian Process

It is a collection of random variables, where any finite number of variables have a joint Gaussian distribution. A Gaussian process (GP) is defined by its mean function $m(x)$ and covariance function $k(x, x')$ as,

$$
f(x) \sim GP(m(x), k(x, x'))
$$


<br>

- **Mean Function**: $m(x) = \mathbb{E}[f(x)]$
<br>
<br>
- **Covariance Function**: $k(x, x') = \mathbb{E}[(f(x) - m(x))(f(x) - m(x'))]$


In [22]:
import numpy as np
import matplotlib.pyplot as plt

# Define mean function and covariance function of the Gaussian process
def mean_function_on_x(x):
    return np.sin(x) + 0.1 * x + 0.01 * x**2

def covariance_function_on_x(x1, x2, length_scale=1.0, amplitude=1.0):
    return amplitude(x1) * np.exp(-0.5 * (np.subtract.outer(x1, x2) / length_scale)**2) * amplitude(x2)

# Define spatially-varying amplitude function for variance
def amplitude_function(x):
    return 0.1 + 0.05 * np.sin(x)


def marginal_Gaussian_process_value_evaluated_on_x(mean, covariance, x, x_value):
    # Calculate the mean and variance of the normal distribution at the given location x_value
    x_index = np.abs(x - x_value).argmin()  # Index of x_value in x
    normal_mean = mean[x_index]
    normal_variance = covariance[x_index, x_index]
    
    x_values = np.linspace(normal_mean - 3 * np.sqrt(normal_variance), normal_mean + 3 * np.sqrt(normal_variance), 100)

    # Calculate the corresponding y values using the normal distribution
    y_values = norm.pdf(x_values, loc=normal_mean, scale=np.sqrt(normal_variance))

    plt.plot( y_values, x_values,color='blue')

    # Customize plot
    plt.title('Normal Distribution at x = {:.2f}'.format(x_value))
    plt.ylim((-0.55, 3.01))
    
# Define the locations where we want to evaluate the Gaussian process
def plot_gaussian_process_evaluated_on_x(x_value):
    x = np.linspace(0, 10, 100)  # Define locations

    # Compute mean and covariance matrices for the given locations
    mean = mean_function_on_x(x)
    covariance = covariance_function_on_x(x, x, amplitude=amplitude_function)

    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [.75, .25]})

    plt.subplot(1, 2, 1)
    # Plot mean function
    plt.plot(x, mean, color='black', label='Mean')
    plt.fill_between(x, mean - 3 * np.sqrt(np.diag(covariance)), mean + 3 * np.sqrt(np.diag(covariance)), color='blue', alpha=0.2, label='Uncertainty')
    plt.title('Gaussian Process with Mean Prediction and Uncertainty')
    plt.axvline(x=x_value, color='r', linestyle='--', label=f'X = {x_value}')
    plt.xlabel('X')
    plt.ylabel('f(X)')
    plt.xlim((0,10))
    plt.ylim((-0.55, 3.01))
    plt.legend()
    
    # Plot conditional probability given X = x_value
    plt.subplot(1, 2, 2)
    marginal_Gaussian_process_value_evaluated_on_x(mean, covariance, x, x_value)

    # Customize plot

    plt.grid(True)


In [23]:
hide_toggle(for_next=True)

In [24]:
style = {'handle_color': 'white', "value":"white"}
widget = interact(plot_gaussian_process_evaluated_on_x,
         x_value=FloatSlider(value=0, min=0, max=10, step=0.3, description='X:', layout=Layout(width='50%'), style= style))

interactive(children=(FloatSlider(value=0.0, description='X:', layout=Layout(width='50%'), max=10.0, step=0.3,…

## Mean Function

<br>

The mean function represents the expected value of the process at any given point. 

- **Zero Mean Function**: The simplest assumption is to assume that it is zero everywhere, i.e., $ m(x) = 0 $ for all $ x $. 

<br>

- **Non-Zero Mean Function**: In some cases, prior knowledge or domain expertise may suggest a non-zero mean function. 

In [25]:
import numpy as np
from scipy.special import kv
from scipy.special import gamma
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Math

def mean_sin(x):
    return np.sin(x)

def mean_linear(x):
    return x*(1/4)


def mean_zero(x):
    return np.zeros_like(x)


class kernel_matern:
    def __init__(self, length_scale=1.0, nu=0.5, amplitude=1.0):
        self.length_scale = length_scale
        self.nu = nu
        self.amplitude = amplitude
    
    def print_formula(self):
        eqn = r"k_{\text{Matern}}(x, x') = \sigma^2\frac{2^{1-\nu}}{\Gamma(\nu)} \left(\frac{\sqrt{2\nu}}{l} | x - x' | \right)^\nu K_\nu \left(\frac{\sqrt{2\nu}}{l} | x - x' | \right)"
        display(Math(r"\text{Matérn Kernel:}"))
        display(Math(eqn))
    
    def compute(self, x1, x2):
        dist = np.sqrt((x1[:, None] - x2[None, :])**2)
        dist[dist == 0.0] += np.finfo(float).eps
        term1 = 2**(1 - self.nu) / gamma(self.nu)
        term2 = np.sqrt(2 * self.nu) * dist / self.length_scale
        term3 = kv(self.nu, term2)
        return self.amplitude * term1 * (term2 ** self.nu) * term3
    
    def update_ls(self, ls):
        self.length_scale = ls
    
    def update_nu(self, nu):
        self.nu = nu
    
    def update_amplitude(self, amp):
        self.amplitude = amp

        
class kernel_gaussian():
    def __init__(self,length_scale=1.0, amplitude=1.0):
        self.length_scale = length_scale
        self.amplitude = amplitude
    
    def print_formula(self):
        eqn = r"""
k_{\text{RBF}}(x, x') = \exp\left(-\frac{(x - x')^2}{2l^2}\right)
"""
        display(Math(r"\text{RBF Kernel:}"))
        display(Math(eqn))
        
    def compute(self, x1, x2):
         return self.amplitude * np.exp(-0.5 * ((x1[:, None] - x2[None, :]) / self.length_scale)**2)
    
    def update_ls(self,ls):
        self.length_scale = ls
    
    def update_amplitude(self,amp):
        self.amplitude = amp
        
class kernel_linear():
    def __init__(self,intersect= 1.0, slope=1/4):
        self.slope = slope
        self.intersect = intersect
    
    def print_formula(self):
        eqn = r"""
k_{\text{Linear}}(x, x') = (x - c) (x' - c)\sigma^2_v + \sigma^2_b
"""
        display(Math(r"\text{Periodic Kernel:}"))
        display(Math(eqn))
            
    def compute(self, x1, x2):
         return self.intersect + self.slope * (x1[:, None] ) *(x2[None, :] )
    
    def update_slope(self,slope):
        self.slope = slope
    
    def update_intersect(self,intersect):
        self.intersect = intersect
        
class kernel_periodic():
    def __init__(self,length_scale=1.0, period=1.0, amplitude=1.0):
        self.length_scale = length_scale
        self.period = period
        self.amplitude = amplitude
    
    def print_formula(self):
        eqn = r"""
k_{\text{Periodic}}(x, x') = \sigma^2\exp\left(-\frac{2\sin^2(\pi | x - x' | / p)}{\ell^2}\right)
"""
        display(Math(r"\text{Periodic Kernel:}"))
        display(Math(eqn))
        
    def compute(self, x1, x2):
         return self.amplitude * np.exp(-2 * (np.sin(np.pi * np.abs(x1[:, None] - x2[None, :]) / self.period) / self.length_scale)**2)
    
    def update_ls(self,ls):
        self.length_scale = ls
    
    def update_amplitude(self,amp):
        self.amplitude = amp
    
    def update_period(self,period):
        self.period = period
        
class kernel_white_noise():
    def __init__(self,noise=1.0):
        self.noise =noise
    
    def print_formula(self):
        eqn = r"""
k_{\text{WhiteNoise}}(x, x') = \sigma^2 \delta(x, x')
"""
        display(Math(r"\text{White Noise Kernel:}"))
        display(Math(eqn))

    def compute(self, x1, x2):
        return np.identity(len(x1)) * self.noise
    
    def update_noise(self, noise):
        self.noise = noise

def plot_gaussian_process(mean_function, covariance_function, num_samples=5):
    # Define the locations where we want to evaluate the Gaussian process
    x = np.linspace(0, 10, 80)  # Define locations

    # Compute mean and covariance matrices for the given locations
    mean = mean_function(x)
    covariance = covariance_function(x, x)
    # Sample from the Gaussian process distribution
    samples = np.random.multivariate_normal(mean, covariance, size=num_samples)

#     # Plotting
#     plt.figure(figsize=(10, 6))

    # Plot mean function
    plt.plot(x, mean, color='black', linestyle='--', label='Mean')

    # Plot uncertainty (two standard deviations above and below the mean)
    plt.fill_between(x, mean - 2 * np.sqrt(np.diag(covariance)), mean + 2 * np.sqrt(np.diag(covariance)), color='blue', alpha=0.2, label='Uncertainty')

    # Plot realizations
    for i in range(num_samples):
        plt.plot(x, samples[i], label=f'Realization {i+1}')

    # Customize plot
    plt.title('Gaussian Process with Mean Prediction and Uncertainty')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim((0,10))
    plt.ylim((-4,4))
    plt.legend()
    plt.grid(True)
    
def plot_covariance_matrix(covariance_function):
    x = np.linspace(0, 10, 80)  # Define locations
    covariance = covariance_function(x, x)
    sn.heatmap(covariance, cmap="coolwarm", xticklabels=False, yticklabels=False)

    # Add title and labels
    plt.title("Kernel Matrix")
    plt.xlabel(r"$x$", size=16)
    plt.ylabel(r"$x'$", size=16)

    

In [26]:
hide_toggle(for_next=True)

In [27]:
# Create an output widget
output = widgets.Output()

# Create buttons for mean functions
button_sin = widgets.Button(description='Sin Mean Function')
button_linear = widgets.Button(description='Linear Mean Function')

# Define callback functions for buttons
def on_click_sin(b):
    with output:
        output.clear_output(wait=True)
        covariance_function = kernel_gaussian()
        plot_gaussian_process(mean_sin, covariance_function.compute)
        plt.show()
def on_click_linear(b):
    with output:
        output.clear_output(wait=True)
        covariance_function = kernel_gaussian()
        plot_gaussian_process(mean_linear, covariance_function.compute)
        plt.show()
# Assign callback functions to buttons
button_sin.on_click(on_click_sin)
button_linear.on_click(on_click_linear)

# Display buttons and output widget
display(widgets.VBox([button_sin, button_linear, output]))

# Trigger the default output
on_click_sin(None)

## Covariance Function

$\textbf{Define}$:
- Similarity/correlation between data points
- Smoothness
- Periodicity

$\textbf{Properties}$:
 - Symmetric
 - Positive Semi-definite

In [28]:
hide_toggle(for_next=True)

In [29]:
# Create buttons for mean and kernel functions
button_kernel_gaussian = widgets.Button(description='Gaussian Kernel')
button_kernel_gaussian.style.text_color = "black"
button_kernel_linear = widgets.Button(description='Linear Kernel')
button_kernel_periodic = widgets.Button(description='Periodic Kernel')
button_kernel_white_noise = widgets.Button(description='White Noise Kernel')
button_kernel_matern = widgets.Button(description='Matern Kernel')
# Create an output widget
output2 = widgets.Output()

kernel_gaussian_instance = kernel_gaussian()
kernel_linear_instance = kernel_linear()
kernel_periodic_instance = kernel_periodic()
kernel_white_noise_instance = kernel_white_noise()
kernel_matern_instance = kernel_matern()
# Define callback functions for buttons

def on_click_kernel_matern(b):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_matern_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_matern_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix( kernel_matern_instance.compute)
        plt.show()
def on_click_kernel_gaussian(b):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_gaussian_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_gaussian_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix( kernel_gaussian_instance.compute)
        plt.show()
        
def on_click_kernel_linear(b):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_linear_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_linear_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix( kernel_linear_instance.compute)
        plt.show()
        
def on_click_kernel_periodic(b):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_periodic_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_periodic_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix( kernel_periodic_instance.compute)
        plt.show()
        
def on_click_kernel_white_noise(b):
    with output2:
        output2.clear_output(wait=True)
        kernel_white_noise_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_white_noise_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_white_noise_instance.compute)
        plt.show()
        
def matern_lengtscale_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_matern_instance.print_formula()
        kernel_matern_instance.update_ls(hyper)
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_matern_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_matern_instance.compute)
        plt.show()
        
def matern_amplitude_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_matern_instance.print_formula()
        kernel_matern_instance.update_amplitude(hyper)
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_matern_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_matern_instance.compute)
        plt.show()
        
def matern_nu_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_matern_instance.print_formula()
        kernel_matern_instance.update_nu(hyper)
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_matern_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_matern_instance.compute)
        plt.show()
        
def rbf_lengtscale_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_gaussian_instance.update_ls(hyper)
        kernel_gaussian_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_gaussian_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_gaussian_instance.compute)
        plt.show()
        
def rbf_amplitude_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_gaussian_instance.update_amplitude(hyper)
        kernel_gaussian_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_gaussian_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_gaussian_instance.compute)
        plt.show()

        
def linear_slope_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_linear_instance.update_slope(hyper)
        kernel_linear_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_linear_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_linear_instance.compute)
        plt.show()
        
def linear_intersect_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_linear_instance.update_intersect(hyper)
        kernel_linear_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_linear_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_linear_instance.compute)
        plt.show()

def periodic_amplitude_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_periodic_instance.update_amplitude(hyper)
        kernel_periodic_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_periodic_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_periodic_instance.compute)
        plt.show()
        
        
def periodic_period_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_periodic_instance.update_period(hyper)
        kernel_periodic_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_periodic_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_periodic_instance.compute)
        plt.show()

def periodic_ls_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_periodic_instance.update_ls(hyper)
        kernel_periodic_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_periodic_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_periodic_instance.compute)
        plt.show()

def white_noise_noise_changed(hyper):
    with output2:
        np.random.seed(0)
        output2.clear_output(wait=True)
        kernel_white_noise_instance.update_noise(hyper)
        kernel_white_noise_instance.print_formula()
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel_white_noise_instance.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel_white_noise_instance.compute)
        plt.show()
        
# Assign callback functions to buttons
button_kernel_gaussian.on_click(on_click_kernel_gaussian)
rbf_lengtscale = widgets.FloatText(value=1, description=r'$l$', layout=Layout(width='50%'))
rbf_lengtscale.observe(lambda event, index: rbf_lengtscale_changed(event['new']), names='value')
rbf_amplitude = widgets.FloatText(value=1, description=r'$\sigma^2$', layout=Layout(width='50%'))
rbf_amplitude.observe(lambda event, index: rbf_amplitude_changed(event['new']), names='value')

button_kernel_linear.on_click(on_click_kernel_linear)
linear_slope = widgets.FloatText(value=1/4, description=r'$\sigma^2_v$', layout=Layout(width='50%'))
linear_slope.observe(lambda event, index: linear_slope_changed(event['new']), names='value')
linear_intersect = widgets.FloatText(value=1, description=r'$\sigma^2_b$', layout=Layout(width='50%'))
linear_intersect.observe(lambda event, index: linear_intersect_changed(event['new']), names='value')

button_kernel_periodic.on_click(on_click_kernel_periodic)
periodic_amplitude =  widgets.FloatText(value=1, description=r'$\sigma^2$', layout=Layout(width='50%'))
periodic_amplitude.observe(lambda event, index: periodic_amplitude_changed(event['new']), names='value')
periodic_lengtscale = widgets.FloatText(value=1, description=r'$l$', layout=Layout(width='50%'))
periodic_lengtscale.observe(lambda event, index: periodic_ls_changed(event['new']), names='value')
periodic_period = widgets.FloatText(value=1, description=r'$p$', layout=Layout(width='50%'))
periodic_period.observe(lambda event, index: periodic_period_changed(event['new']), names='value')

button_kernel_white_noise.on_click(on_click_kernel_white_noise)
white_noise_noise =  widgets.FloatText(value=1, description=r'$\sigma^2$', layout=Layout(width='50%'))
white_noise_noise.observe(lambda event, index: white_noise_noise_changed(event['new']), names='value')


button_kernel_matern.on_click(on_click_kernel_matern)
matern_lengtscale = widgets.FloatText(value=1, description=r'$l$', layout=Layout(width='50%'))
matern_lengtscale.observe(lambda event, index: matern_lengtscale_changed(event['new']), names='value')
matern_amplitude = widgets.FloatText(value=1, description=r'$\sigma^2$', layout=Layout(width='50%'))
matern_amplitude.observe(lambda event, index: matern_amplitude_changed(event['new']), names='value')
options = [1/2, 3/2, 5/2]
matern_nu = widgets.Dropdown(options=options, description=r'$\nu$', value=options[0], layout=Layout(width='60%'))
matern_nu.observe(lambda event, index: matern_nu_changed(event['new']), names='value')

# Display buttons and output widget
# display(widgets.VBox([button_kernel_gaussian, button_kernel_linear, button_kernel_periodic, button_kernel_white_noise, output2]))
widget1 =widgets.VBox([button_kernel_matern, button_kernel_periodic, button_kernel_gaussian, button_kernel_linear, button_kernel_white_noise])
widget2 =widgets.VBox([matern_lengtscale,  periodic_lengtscale, rbf_lengtscale, linear_slope, white_noise_noise])
widget3 =widgets.VBox([matern_amplitude, periodic_amplitude, rbf_amplitude, linear_intersect])
widget4 =widgets.VBox([matern_nu, periodic_period])
interactive_buttons = widgets.HBox([widget1, widget2, widget3, widget4])

display(widgets.VBox([interactive_buttons, output2]))
on_click_kernel_matern(None)

## Combining Kernels


- $\bf Summing\text{ } Kernels$: The resulting covariance allows to capture various patterns simultaneously.

$$
k_{\text{sum}}(x, x') = k_1(x, x') + k_2(x, x') + \dots + k_n(x, x')
$$


- $\bf Multiplying \text{ }Kernels$: This approach is useful for modeling interactions between different patterns present in the data. 


$$
k_{\text{mult}}(x, x') = k_1(x, x') \times k_2(x, x') \times \dots \times k_n(x, x')
$$

In [30]:
hide_toggle(for_next=True)

In [31]:
output_operation = widgets.Output()

class addition_kernel():
    def __init__(self, kernel1, kernel2):
        self.kernel1 = kernel1
        self.kernel2 = kernel2
    
    def compute(self,x1, x2):
        return self.kernel1.compute(x1, x2) + self.kernel2.compute(x1, x2)

    
class multiplication_kernel():
    def __init__(self, kernel1, kernel2):
        self.kernel1 = kernel1
        self.kernel2 = kernel2
    
    def compute(self,x1, x2):
        return self.kernel1.compute(x1, x2) * self.kernel2.compute(x1, x2)
    
def kernel_operation_changed(operation):
    with output_operation:
        np.random.seed(0)
        output_operation.clear_output(wait=True)
        if operation == "addition":
            kernel = addition_kernel(kernel_linear(), kernel_periodic())
        else:
            kernel = multiplication_kernel(kernel_linear(), kernel_periodic())
        display(Math(r"Operation(k_{Linear}, k_{Periodic})"))
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        plt.subplot(1, 2, 2)
        plot_gaussian_process(mean_zero, kernel.compute)
        plt.subplot(1, 2, 1)
        plot_covariance_matrix(kernel.compute)
        plt.show()


options = ["addition", "multiplication"]
operation_widget = widgets.Dropdown(options=options, description=r'Operation', value=options[0], layout=Layout(width='60%'))
operation_widget.observe(lambda event, index: kernel_operation_changed(event['new']), names='value')
widget_operation =widgets.VBox([operation_widget, output_operation])
display(widgets.VBox([widget_operation]))
kernel_operation_changed("addition")

# Predictive Distribution

Given the noise-free observations $ \mathbf{f} $, the joint distribution of observed data and the function values at the test point $ X^* $ can be represented as follows:

$$
\begin{bmatrix}
\mathbf{f} \\
\mathbf{f}^*
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
\textbf{m}\\
\mathbf{m}^*
\end{bmatrix},
\begin{bmatrix}
K(X,X) & K(X, X^*) \\
K(X^*, X) & K(X^*, X^*) 
\end{bmatrix}
\right)
$$

The predictive distribution is obtained by conditioning on the observed data:

$$
\mathbf{f}^* | \mathbf{f} \sim \mathcal{N}(\mu^*, \Sigma^*)
$$

$$ \mu^* = \textbf{m}^* + K(X^*, X)^T K(X,X)^{-1} (\mathbf{f} - \mathbf{m}) $$

$$ \Sigma^* = K(X^*,X^*) - K(X^*, X)^T K(X,X)^{-1} K(X, X^*) $$ 


# Predictive Distribution using Noisy Observation


Consider, $
y = f(x) + \epsilon,\text{ where }\epsilon \sim \mathcal{N}(0, \sigma_{\nu}^2)$

Therefore, the joint distribution is,


$$
\begin{bmatrix}
\mathbf{y} \\
\mathbf{f}^*
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
\textbf{m}\\
\mathbf{m}^*
\end{bmatrix},
\begin{bmatrix}
K(X,X) +  \sigma_{\nu}^2 & K(X, X^*) \\
K(X^*, X) & K(X^*, X^*) 
\end{bmatrix}
\right)
$$

and,

$$
\mathbf{f}^* | \mathbf{y} \sim \mathcal{N}(\mu^*, \Sigma^*)
$$
$$
\mu^* = \textbf{m}^* + K(X^*, X)^T [K(X,X) + \sigma_{\nu}^2 I]^{-1} (\mathbf{y} - \mathbf{m}) 
$$
$$
\Sigma^* = K(X^*,X^*) - K(X^*, X)^T [K(X,X) + \sigma_{\nu}^2 I]^{-1}  K(X, X^*)
$$

In [32]:
import numpy as np

class GaussianProcessRegressor:
    def __init__(self, kernel, sigma_noise=1e-6):
        self.kernel = kernel
        self.sigma_noise = sigma_noise

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.K = self.kernel(self.X_train, self.X_train)
        self.K += np.eye(len(self.X_train)) * self.sigma_noise  # Adding noise to the diagonal for numerical stability
        self.K_inv = np.linalg.inv(self.K)

    def predict(self, X_test):
        K_s = self.kernel(self.X_train, X_test)
        mu_s = K_s.T.dot(self.K_inv).dot(self.y_train)
        cov_s = self.kernel(X_test, X_test) - K_s.T.dot(self.K_inv).dot(K_s)
        return mu_s.reshape(-1), np.sqrt(np.diag(cov_s))

def underlying_function(x):
    return np.sin(x) + np.sin((10.0 / 3.0) * x)


def plot_gaussian_process_with_changing_hypers(x_value, ls, sigma, noise_variance):
    
    # Define a simple RBF kernel
    def rbf_kernel(X1, X2, length_scale=ls, sigma_f=sigma):
        sq_dist = np.sum(X1**2, axis=1).reshape(-1, 1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
        return sigma_f**2 * np.exp(-0.5 / length_scale**2 * sq_dist)

    X_train_existing = np.array([-2, -1, 1, 2]).reshape(-1, 1)

    new_x_value = np.array([[x_value]])

    # Concatenate the new x_value with the existing X_train array
    X_train = np.concatenate([X_train_existing, new_x_value], axis=0)
    # Define some training data
    
    y_train = underlying_function(X_train)

    # Define test data
    X_test_base = np.linspace(-3, 3, 100).reshape(-1, 1)
    X_test = np.sort(np.concatenate([X_test_base, new_x_value], axis=0), axis=0)
    Y_true_f = underlying_function(X_test)
    # Create Gaussian Process Regressor
    gp = GaussianProcessRegressor(kernel=rbf_kernel, sigma_noise=noise_variance)

    # Fit the model
    gp.fit(X_train, y_train)

    # Make predictions
    y_pred_mean, y_pred_std = gp.predict(X_test)

    data = np.concatenate([X_train, y_train], axis=1)
    df_train = pd.DataFrame(data, columns=['X', 'Y'])

    data = np.concatenate([X_test, y_pred_mean.reshape(-1, 1), (y_pred_mean - y_pred_std).reshape(-1, 1), (y_pred_mean + y_pred_std).reshape(-1, 1)], axis=1)
    df_pred = pd.DataFrame(data, columns=['X', 'Predicted Mean', 'Lower Bound', 'Upper Bound'])

    # Plot the results using Seaborn
    plt.figure(figsize=(10, 6))
    sn.scatterplot(x='X', y='Y', data=df_train, color='black', label='Training Data')
    sn.lineplot(x='X', y='Predicted Mean', data=df_pred, color='blue', label='Predicted Mean')
    plt.fill_between(df_pred['X'], df_pred['Lower Bound'], df_pred['Upper Bound'], color='black', alpha=0.2, label='Uncertainty')
    plt.axvline(x=x_value, color='r', linestyle='--', label=f'X = {x_value}')
    plt.plot(X_test, Y_true_f)
    plt.xlim((-3, 3))
    plt.ylim((-2,2))
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Gaussian Process Regression')
    plt.legend()
    plt.show()

In [33]:
hide_toggle(for_next=True)

In [34]:
style = {'handle_color': 'white', "value":"white"}
widget = interact(plot_gaussian_process_with_changing_hypers,
         x_value=FloatSlider(value=0, min=-3, max=3, step=0.001, description='$x_{new}$:', layout=Layout(width='50%'), style= style),
         ls=FloatSlider(value=1, min=1e-3, max=3, step=0.1, description='$l$:', layout=Layout(width='50%'), style= style),
        sigma=FloatSlider(value=1, min=1e-3, max=20, step=0.1, description='$\sigma_k$:', layout=Layout(width='50%'), style= style),
        noise_variance=FloatSlider(value=1e-6, min=1e-6, max=2, step=0.01, description='$\sigma^2_{\nu}$:', layout=Layout(width='50%'), style= style))

interactive(children=(FloatSlider(value=0.0, description='$x_{new}$:', layout=Layout(width='50%'), max=3.0, mi…

# Learning Hyperparameters

Given the marginal likelihood of the observed data.

$$
p(\mathbf{f}^* | X, \mathbf{y}, X^*, \theta) = \frac{p(\mathbf{y} , \mathbf{f}^*| X^*, X, \theta)}{\underbrace{p(\mathbf{y} | \mathbf{X}, \theta)}_{\text{marginal likelihood}}} = \frac{p(\mathbf{y} , \mathbf{f}^*| X^*, X, \theta)}{ \int p(\mathbf{y} | \mathbf{f}) p(\mathbf{f} | X, \theta) d\mathbf{f} }
$$

We maximize the marginal likelihood with respect to the hyperparameters $\theta$. 

$$
\theta^* = \text{argmax}_\theta \left( -\log p(\mathbf{y} | X, \theta) \right)
$$

where, $\log p(y | X, \theta) = -\frac{1}{2} \left( \mathbf{y}^\top (K(X, X) + \sigma_n^2 I)^{-1} \mathbf{y} + \log |K(X,X) + \sigma_n^2 I| + n \log (2\pi) \right)$

In [35]:
# Define a simple RBF kernel
def kernel(length_scale, sigma_f):
    def rbf_kernel(X1, X2):
        sq_dist = np.sum(X1**2, axis=1).reshape(-1, 1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
        return sigma_f**2 * np.exp(-(0.5 / length_scale**2) * sq_dist)
    return rbf_kernel

def marginal_likelihood(X, y):
    def compute_log_marginal_likelihood(length_scale, noise_level):
        rbf_kernel = kernel(length_scale, sigma_f=1)
        K = rbf_kernel(X, X) + noise_level**2 * np.eye(len(X))
        # Cholesky decomposition of the covariance matrix
        L = np.linalg.cholesky(K + 1e-6 * np.eye(len(X)))
        # Solve for alpha: K * alpha = y
        alpha = np.linalg.solve(L.T, np.linalg.solve(L, y))
        # Compute log determinant of the covariance matrix
        log_det_K = 2.0 * np.sum(np.log(np.diag(L)))
        # Compute log marginal likelihood
        log_marginal_likelihood = -0.5 * ((y.T).dot(alpha) + log_det_K + len(X) * np.log(2 * np.pi))
        return log_marginal_likelihood
    return compute_log_marginal_likelihood

def plot_marginal_likelihood_at_location(length_scale, noise_level):
    np.random.seed(19)
    lb = -2.5
    ub = 2.5
    X = (np.random.random(4)*(ub - lb) + lb).reshape(-1, 1)
    y = underlying_function(X) + np.random.random(len(X)).reshape(-1,1)*0.3
    log_marginal_likelihood = marginal_likelihood(X, y)

    # Define the range of hyperparameters
    noise_levels = np.linspace(1e-6, 1.0, 60)
    length_scales = np.linspace(0.1, 1.0, 20)

    # Create a grid of hyperparameter values
    noise_levels_grid, length_scales_grid = np.meshgrid(noise_levels, length_scales)

    # Compute the log marginal likelihood for each combination of hyperparameters
    log_likelihood_values = np.zeros_like(noise_levels_grid)
    for i in range(len(length_scales)):
        for j in range(len(noise_levels)):
            log_likelihood_values[i, j] = np.exp(log_marginal_likelihood(length_scales[i], noise_levels[j]))

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    plt.subplot(1, 2, 1)

    sn.set_style("darkgrid")
    # Plot the landscape of the log marginal likelihood
    plt.contourf(noise_levels_grid, length_scales_grid, log_likelihood_values, levels=50)
    plt.contour(noise_levels_grid, length_scales_grid, log_likelihood_values, levels=50, colors='black')
    plt.scatter(noise_level, length_scale, marker="x", zorder=20)
    plt.xlabel('Noise Level')
    plt.ylabel('Length Scale')
    plt.title('Marginal Likelihood Landscape')
    
    plt.subplot(1, 2, 2)
     # Define test data
    X_train = X
    y_train = y
    X_test = np.linspace(-3, 3, 60).reshape(-1, 1)
    Y_true_f = underlying_function(X_test)
    # Create Gaussian Process Regressor
    rbf_kernel = kernel(length_scale, sigma_f=1)
    gp = GaussianProcessRegressor(kernel=rbf_kernel, sigma_noise=noise_level)

    # Fit the model
    gp.fit(X_train, y_train)

    # Make predictions
    y_pred_mean, y_pred_std = gp.predict(X_test)

    data = np.concatenate([X_train, y_train], axis=1)
    df_train = pd.DataFrame(data, columns=['X', 'Y'])

    data = np.concatenate([X_test, y_pred_mean.reshape(-1, 1), (y_pred_mean - y_pred_std).reshape(-1, 1), (y_pred_mean + y_pred_std).reshape(-1, 1)], axis=1)
    df_pred = pd.DataFrame(data, columns=['X', 'Predicted Mean', 'Lower Bound', 'Upper Bound'])

    # Plot the results using Seaborn
    sn.scatterplot(x='X', y='Y', data=df_train, color='black', label='Training Data')
    sn.lineplot(x='X', y='Predicted Mean', data=df_pred, color='blue', label='Predicted Mean')
    plt.fill_between(df_pred['X'], df_pred['Lower Bound'], df_pred['Upper Bound'], color='black', alpha=0.2, label='Uncertainty')
    plt.plot(X_test, Y_true_f)
    plt.xlim((-3, 3))
    plt.ylim((-2,2))
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Gaussian Process Regression')
    plt.legend()

In [36]:
hide_toggle(for_next=True)

In [37]:
style = {'handle_color': 'white', "value":"white"}
widget = interact(plot_marginal_likelihood_at_location,
         length_scale=FloatSlider(value=0.58, min=0.1, max=1, step=0.1, description='$l_s$', layout=Layout(width='50%'), style= style),
        noise_level=FloatSlider(value=0.32, min=1e-6, max=1, step=0.1, description='$\sigma_{\nu}^2$', layout=Layout(width='50%'), style= style))

interactive(children=(FloatSlider(value=0.58, description='$l_s$', layout=Layout(width='50%'), max=1.0, min=0.…

# Difficulties

- Presents a $\mathcal{O}(n^3)$ computational cost and $\mathcal{O}(n^2)$ in memory.
    - Sparse approximation.
    
- Challenging in high number of dimensions.
    - Structural assumptions
    
- The marginal likelihood is often multi-modal, i.e, local optima.
    -  Random start points, using prior distributions, marginalise over hyperparameters.

# RISE configuration

In [38]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
tmp = cm.update(
        "rise",
        {
            "theme": "simple",
            "transition": "fade",
            "start_slideshow_at": "selected",
            "autolaunch": True,
            "width": "100%",
            "height": "100%",
            "header": "",
            "footer":"",
            "scroll": True,
            "enable_chalkboard": True,
            "slideNumber": True,
            "center": False,
            "controlsLayout": "edges",
            "slideNumber": True,
            "hash": True,
        }
    )